In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from keras import regularizers

Using TensorFlow backend.


In [2]:
train_data = pd.read_csv("data/train.csv", index_col=False)
test_data = pd.read_csv("data/test.csv", index_col=False)
train_data.shape, test_data.shape

((1460, 81), (1459, 80))

In [3]:
tr_id = train_data["Id"]
te_id = test_data["Id"]
tr_y = np.log1p(train_data["SalePrice"])

In [4]:
train_data.drop("Id", axis = 1, inplace = True)
test_data.drop("Id", axis = 1, inplace = True)

In [5]:
ntrain = train_data.shape[0]
ntest = test_data.shape[0]
data = pd.concat((train_data, test_data)).reset_index(drop=True)
data.drop(['SalePrice'], axis=1, inplace=True)

In [6]:
data["PoolQC"] = data["PoolQC"].fillna("None")
data["MiscFeature"] = data["MiscFeature"].fillna("None")
data["Alley"] = data["Alley"].fillna("None")
data["Fence"] = data["Fence"].fillna("None")
data["FireplaceQu"] = data["FireplaceQu"].fillna("None")
data["LotFrontage"] = data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    data[col] = data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    data[col] = data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    data[col] = data[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    data[col] = data[col].fillna('None')
data["MasVnrType"] = data["MasVnrType"].fillna("None")
data["MasVnrArea"] = data["MasVnrArea"].fillna(0)
data['MSZoning'] = data['MSZoning'].fillna(data['MSZoning'].mode()[0])
data = data.drop(['Utilities'], axis=1)
data["Functional"] = data["Functional"].fillna("Typ")
data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])
data['KitchenQual'] = data['KitchenQual'].fillna(data['KitchenQual'].mode()[0])
data['Exterior1st'] = data['Exterior1st'].fillna(data['Exterior1st'].mode()[0])
data['Exterior2nd'] = data['Exterior2nd'].fillna(data['Exterior2nd'].mode()[0])
data['SaleType'] = data['SaleType'].fillna(data['SaleType'].mode()[0])
data['MSSubClass'] = data['MSSubClass'].fillna("None")
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(data[c].values)) 
    data[c] = lbl.transform(list(data[c].values))
print('Shape all_data: {}'.format(data.shape))

Shape all_data: (2919, 78)


In [7]:
dummy_data = pd.get_dummies(data)
tr_df = dummy_data[:ntrain]
te_df = dummy_data[ntrain:]

In [8]:
tr_df.shape

(1460, 220)

In [14]:
# define wider model
def wider_model():
	# create model
	model = Sequential()
	model.add(Dense(256, input_dim=220, kernel_initializer='normal', activation='selu',
                    kernel_regularizer=regularizers.l2(0.00001),
                activity_regularizer=regularizers.l1(0.00001)))
	model.add(Dense(64, kernel_initializer='normal', activation='selu'))
	model.add(Dense(64, kernel_initializer='normal', activation='selu'))
	model.add(Dense(32, kernel_initializer='normal', activation='selu'))
	model.add(Dense(32, kernel_initializer='normal', activation='selu'))
	model.add(Dense(16, kernel_initializer='normal', activation='selu'))
	model.add(Dense(16, kernel_initializer='normal', activation='selu'))
	model.add(Dense(8, kernel_initializer='normal', activation='selu'))
	model.add(Dense(4, kernel_initializer='normal', activation='selu'))
	model.add(Dense(1))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [15]:
sts = StandardScaler().fit(dummy_data.values)
tr_X = sts.transform(tr_df.values)
te_X = sts.transform(te_df.values)

In [19]:
estimator = KerasRegressor(build_fn=wider_model, epochs=250, batch_size=64, verbose=1)
estimator.fit(tr_X, tr_y.values)

Epoch 1/250
1460/1460 [==============================] - 0s - loss: 140.6768     
Epoch 2/250
1460/1460 [==============================] - 0s - loss: 105.1189     
Epoch 3/250
1460/1460 [==============================] - 0s - loss: 42.1402     
Epoch 4/250
1460/1460 [==============================] - 0s - loss: 2.1266     

In [20]:
score = mean_squared_error(tr_y.values, estimator.predict(tr_X))
print("\n",score)

  64/1460 [>.............................] - ETA: 2s
 0.0401755173796


In [21]:
te_y = np.expm1(estimator.predict(te_X))
import time
res = pd.DataFrame({"Id": te_id, "SalePrice": te_y.reshape(te_y.shape[0])})
res.to_csv("results/linear_%s.csv" %time.time(), index=False)

  64/1459 [>.............................] - ETA: 0s